# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

data = load_boston()
X = data.data
y = data.target

X_train = X[:int(0.75*X.shape[0])+1]
X_test = X[int(0.75*X.shape[0])+1:]
y_train = y[:int(0.75*X.shape[0])+1]
y_test = y[int(0.75*X.shape[0])+1:]

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [2]:
def antigrad(y, y_pred):
    return y - y_pred

def write_answer(value, number):
    with open("answer_"+str(number)+".txt", "w") as fout:
        fout.write(str(value))

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

size = 50
base_algorithms_list = []
coefficients_list = []

for i in range(size):
    y_pred = gbm_predict(X_train)
    tree = DecisionTreeRegressor(max_depth=5, random_state=42)
    tree.fit(X_train, antigrad(y_train, y_pred))
    base_algorithms_list.append(tree)
    coefficients_list.append(0.9)
    print(mean_squared_error(y_test, gbm_predict(X_test)))

write_answer(mean_squared_error(y_test, gbm_predict(X_test))**0.5, 2)

21.850672141336073
26.437263475077273
26.25588457053095
27.483150744849887
28.388457215540548
29.644668733848643
30.078933232320658
30.255060365741965
29.654122395575772
29.828816686895067
29.795494886741906
29.828524638221186
29.5608852683092
29.481888741742555
30.16992049224153
30.07528623702009
29.852946618710543
29.832666971259428
29.952154318215225
29.62436532919044
29.753837111678894
29.768298569779986
29.834735114723124
29.900404902616074
29.876932062182032
29.84553903175389
29.87016061710371
29.77528463566106
29.800432241880745
29.803189415976107
29.8066589715236
29.807901442475515
29.79001644125618
29.801591126941513
29.745890834209863
29.747618860552468
29.74741053299291
29.74702379836982
29.748946408007228
29.746948343579685
29.74542435187091
29.75893955480756
29.76162412608486
29.76154747424801
29.763801425884143
29.76724202941968
29.767559042641302
29.764330498998827
29.762223037117355
29.762175555949725


## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [4]:
base_algorithms_list = []
coefficients_list = []

for i in range(size):
    y_pred = gbm_predict(X_train)
    tree = DecisionTreeRegressor(max_depth=5, random_state=42)
    tree.fit(X_train, antigrad(y_train, y_pred))
    base_algorithms_list.append(tree)
    coefficients_list.append(0.9/(1.0 + i))
    print(mean_squared_error(y_test, gbm_predict(X_test)))

write_answer(mean_squared_error(y_test, gbm_predict(X_test))**0.5, 3)

21.850672141336073
23.631740558478366
24.60801248224706
24.790372648895875
25.135181180163144
25.702362723945217
25.952920231356355
26.03833258542793
26.03798169955581
26.139172865578505
26.29089251598051
26.369647113282028
26.41369527385452
26.469129517903113
26.512578168992405
26.571909022772324
26.576111419128527
26.714496211792817
26.777982098406294
26.865644856155082
26.80617470202417
26.933578871311035
26.897589020837774
26.996379687159006
27.029804872817227
27.09963129881996
27.14662291865034
27.084739455052638
27.144512620366537
27.18446786946045
27.158568227150393
27.228841877712334
27.252948583759135
27.261805524346002
27.260839063180384
27.29393077824865
27.293489787815435
27.336743338717092
27.353264631217744
27.355896881324966
27.361525346880683
27.37816374956198
27.392494421659194
27.39613886684019
27.440963231328155
27.440797145497896
27.45482487351274
27.456425520708247
27.44934437039549
27.46538283329386


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [5]:
value = "2 3"
write_answer(value, 4)

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [6]:
from sklearn.linear_model import LinearRegression

In [7]:
reg = LinearRegression()
reg.fit(X_train, y_train)
error = mean_squared_error(y_test, reg.predict(X_test))**0.5
print(error)
write_answer(error, 5)

7.848121796479828
